In [5]:
import sys
import os
from pathlib import Path
sys.path.append(os.getcwd())
sys.path.append(str(Path(os.getcwd()).parent))
import torch
import pprint
from utils import *
from configuration import *
from NEAT_args import *

args = parser.parse_args([])
EA_args = EA_parser.parse_args([])

In [18]:
import re

results = torch.zeros([13,10,3])

args.DEVICE = 'cpu'

for ds in range(13):
    args.DATASET = ds
    args = FormulateArgs(args)
    valid_loader, datainfo = GetDataLoader(args, 'valid', path='../dataset/')
    test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset/')
    pprint.pprint(datainfo)
    
    for seed in range(10):
        args.SEED = seed
        for e, eps_train in enumerate([0.0, 0.05, 0.1]):
            args.e_train = eps_train

            logname = f"data:{datainfo['dataname']}_seed:{args.SEED:02d}_epsilon:{eps_train}.model.log"
            logpath = f'./log/{logname}'
            
            if os.path.exists(logpath):

                with open(logpath, 'r', encoding='utf-8') as file:
                    data = file.read()

                epoch_pattern = r"\| Epoch:\s+(\d+)\s+\|\s+Train"
                epoch_matches = re.findall(epoch_pattern, data)
                
                time_pattern = r"Epoch time:\s+(\d+\.\d+)"
                time_matches = re.findall(time_pattern, data)

                epoch_numbers = torch.tensor([int(match) for match in epoch_matches], dtype=torch.int32)
                time_values = torch.tensor([float(time) for time in time_matches], dtype=torch.float32)

                epochs_col = epoch_numbers.unsqueeze(1)
                times_col = time_values.unsqueeze(1)
                result_tensor = torch.cat((epochs_col, times_col), dim=1)

                if result_tensor.numel():
                    mean_epoch_time = result_tensor[:,1].mean()
                    max_epoch = result_tensor[:,0].max()
    
                    train_time = max_epoch * mean_epoch_time / 60

                    if 'Training if finished.' in data:
                        
                        results[ds,seed,e] = train_time
                        


{'N_class': 2,
 'N_feature': 6,
 'N_test': 25,
 'N_train': 70,
 'N_valid': 23,
 'dataname': 'acuteinflammation'}
{'N_class': 3,
 'N_feature': 4,
 'N_test': 79,
 'N_train': 373,
 'N_valid': 124,
 'dataname': 'balancescale'}
{'N_class': 2,
 'N_feature': 9,
 'N_test': 79,
 'N_train': 397,
 'N_valid': 127,
 'dataname': 'breastcancerwisc'}
{'N_class': 3,
 'N_feature': 21,
 'N_test': 79,
 'N_train': 397,
 'N_valid': 127,
 'dataname': 'cardiotocography3clases'}
{'N_class': 3,
 'N_feature': 8,
 'N_test': 79,
 'N_train': 397,
 'N_valid': 127,
 'dataname': 'energyy1'}
{'N_class': 3,
 'N_feature': 8,
 'N_test': 79,
 'N_train': 397,
 'N_valid': 127,
 'dataname': 'energyy2'}
{'N_class': 3,
 'N_feature': 4,
 'N_test': 31,
 'N_train': 88,
 'N_valid': 29,
 'dataname': 'iris'}
{'N_class': 2,
 'N_feature': 5,
 'N_test': 79,
 'N_train': 397,
 'N_valid': 127,
 'dataname': 'mammographic'}
{'N_class': 10,
 'N_feature': 16,
 'N_test': 79,
 'N_train': 397,
 'N_valid': 127,
 'dataname': 'pendigits'}
{'N_class'

In [26]:
results.mean(2)

tensor([[144.9187, 140.1584, 139.6750, 141.1455, 143.6579, 143.4301, 145.7790,
         138.7656, 136.0628, 144.7349],
        [113.9039, 117.3555, 113.3418, 135.6349, 107.3076, 108.8630, 106.7210,
         126.5416, 127.2229, 131.9617],
        [133.4599, 127.8386, 120.0633, 120.4542, 120.2475, 129.6051, 121.8956,
         136.7751, 113.8114, 125.9533],
        [125.8457, 124.5898, 119.8918, 105.3541, 114.8730, 105.4992, 109.6585,
         129.7741, 123.2236, 114.9837],
        [117.8853, 121.0274, 140.3900, 133.5696, 146.9312, 127.4208, 144.5143,
         140.1302, 124.0166, 134.0397],
        [146.7011, 132.6959, 140.9290, 159.8171, 156.1958, 147.3422, 142.6969,
         143.5955, 143.8298, 153.2693],
        [118.5967, 114.0440, 115.8196, 112.1638, 109.5162, 116.5254, 107.3408,
         111.4914, 116.1386, 117.3909],
        [136.6495, 122.7156, 130.8287, 126.8111, 137.0737, 111.9372, 135.8995,
         132.5701, 121.6764, 114.8609],
        [ 93.0896,  81.0813,  88.0878,  90.6571,

In [20]:
re = results.mean([1,2])
re.shape

torch.Size([13])

In [21]:
re

tensor([141.8328, 118.8854, 125.0104, 117.3694, 132.9925, 146.7073, 113.9027,
        127.1023,  91.1831,  78.5866,  84.0392,  80.9016,  90.8558])